# SOLAR HIGH ALTITUDE LONG ENDURANCE AIRCRAFT

Import necessary modules

In [ ]:
from gpkit import Variable, Model, units
from gpkit.constraints.set import ConstraintSet
from gpkit import LinkedConstraintSet
import numpy as np
import matplotlib.pyplot as plt

## Define System Level Variables

In [ ]:
CD = Variable('C_D', '-', 'Drag coefficient')
CL = Variable('C_L', '-', 'Lift coefficient')
P_shaft = Variable('P_{shaft}', 'W', 'Shaft power')
S = Variable('S', 'm^2', 'Wing reference area')
V = Variable('V', 'm/s', 'Cruise velocity')
W = Variable('W', 'lbf', 'Aircraft weight')
rho = Variable(r'\rho', 'kg/m^3')
E_batt = Variable('E_{batt}', 'J', 'Battery energy')
h = Variable("h", "ft", "Altitude")
g = Variable('g', 9.81, 'm/s^2', 'Gravitational acceleration')

## Steady Level Flight

We are assuming steady level flight. Specifically, the required lift is equal to the weight and the shaft power produced by the engine has to be greater than the flight power. 

In [ ]:
class SteadyLevelFlight(Model):
    def __init__(self, **kwargs):
        eta_prop = Variable(r'\eta_{prop}', 0.7, '-', 'Propulsive efficiency')

        constraints = [P_shaft >= V*W*CD/CL/eta_prop,   # eta*P = D*V
                       W == 0.5*rho*V**2*CL*S]
        Model.__init__(self, None, constraints, **kwargs)

## Aerodynamics 

We assumed that it the drag was combination of non-lifting drag, wing profile drag, and induced drag.

In [ ]:
class Aero(Model):
    def __init__(self, **kwargs):

        Cd0 = Variable('C_{d0}', 0.002, '-', "non-wing drag coefficient")
        cdp = Variable("c_{dp}", "-", "wing profile drag coeff")
        CLmax = Variable('C_{L-max}', 1.5, '-', 'maximum lift coefficient')
        e = Variable('e', 0.9, '-', "spanwise efficiency")
        AR = Variable('AR', 27, '-', "aspect ratio")
        b = Variable('b', 'ft', 'span')
        mu = Variable(r'\mu', 1.5e-5, 'N*s/m^2', "dynamic viscosity")
        Re = Variable("Re", '-', "Reynolds number")
        Re_ref = Variable("Re_{ref}", 3e5, "-", "Reference Re for cdp")
        Cf = Variable("C_f", "-", "wing skin friction coefficient")

        constraints = [
            CD >= Cd0 + cdp + CL**2/(np.pi*e*AR),
            cdp >= ((0.006 + 0.005*CL**2 + 0.00012*CL**10)*(Re/Re_ref)**-0.3),
            b**2 == S*AR,
            CL <= CLmax,
            Re == rho*V/mu*(S/AR)**0.5,
            ]
        Model.__init__(self, None, constraints, **kwargs)

## Weights

We assumed that the weight of the airframe, which includes the spar, wing, engines, and fuselage structural weight, is a percentage of the weight of the aircraft. The energy battery density is based on the best available lithium ion batteries. The avionics weight is based on required avionics for flight control, ground communication, and satellite communication.

In [ ]:
class Weight(Model):
    def __init__(self, **kwargs):

        W_batt = Variable('W_{batt}', 'lbf', 'Battery weight')
        W_airframe = Variable('W_{airframe}', 'lbf', 'Airframe weight')
        W_solar = Variable('W_{solar}', 'lbf', 'Solar panel weight')
        W_pay = Variable('W_{pay}', 4, 'lbf', 'Aircraft weight')
        W_avionics = Variable('W_{avionics}', 4, 'lbf', 'avionics weight')
        rho_solar = Variable(r'\rho_{solar}', 1.2, 'kg/m^2',
                             'Solar cell area density')
        f_airframe = Variable('f_{airframe}', 0.20, '-',
                              'Airframe weight fraction')
        h_batt = Variable('h_{batt}', 250, 'W*hr/kg', 'Battery energy density')

        constraints = [W_airframe >= W*f_airframe,
                       W_batt >= E_batt/h_batt*g,
                       W_solar >= rho_solar*g*S,
                       W >= W_pay + W_solar + W_airframe + W_batt + W_avionics]
        Model.__init__(self, None, constraints, **kwargs)

## Power 

The value assumed for the average daytime solar irradiance, was based on standard solar irradiance values.   The accessory power draw is based on the approximate power usage from the avionics (15 Watts) and payload (10 Watts).  The night span is 8 hours and is based on the worst case scenario for the 45 degree latitude at the winter solstice, assuming that if it can flight for the longest night of the year on the power charged during the day then it theoretically flight all year long.  The incidence angle is included in this calculation as the $\cos{\theta}$, where $\theta$ is the incidence angle, because the cosine function is not a convex function.  The solar cell efficiency was assumed to be \%20.  


In [ ]:
class Power(Model):
    def __init__(self, **kwargs):

        PS_irr = Variable('(P/S)_{irr}', 1000*0.5, 'W/m^2',
                          'Average daytime solar irradiance')
        P_oper = Variable('P_{oper}', 'W', 'Aircraft operating power')
        P_charge = Variable('P_{charge}', 'W', 'Battery charging power')
        P_acc = Variable('P_{acc}', 25, 'W', 'Accessory power draw')
        eta_solar = Variable(r'\eta_{solar}', 0.2, '-',
                             'Solar cell efficiency')
        eta_charge = Variable(r'\eta_{charge}', 0.95, '-',
                              'Battery charging efficiency')
        eta_discharge = Variable(r'\eta_{discharge}', 0.95, '-',
                                 'Battery discharging efficiency')
        t_day = Variable('t_{day}', 'hr', 'Daylight span')
        t_night = Variable('t_{night}', 16, 'hr', 'Night span')
        th = Variable(r'\theta', 0.35, '-', 
                      'cosine of incidence angle at 45 lat')

        constraints = [PS_irr*eta_solar*S >= P_oper + P_charge,
                       P_oper >= P_shaft + P_acc,
                       P_charge >= E_batt/(t_day*eta_charge*th),
                       t_day + t_night <= 24*units.hr,
                       E_batt >= P_oper*t_night/eta_discharge]
        Model.__init__(self, None, constraints, **kwargs)

## Atmosphere 

The basic assumption here is that temperature, air density and pressure vary with altitude. 

In [ ]:
class Atmosphere(Model):
    def __init__(self, **kwargs):

        p_sl = Variable("p_{sl}", 101325, "Pa", "Pressure at sea level")
        T_sl = Variable("T_{sl}", 288.15, "K", "Temperature at sea level")
        L_atm = Variable("L_{atm}", 0.0065, "K/m", "Temperature lapse rate")
        T_atm = Variable("T_{atm}", "K", "air temperature")
        M_atm = Variable("M_{atm}", 0.0289644, "kg/mol",
                         "Molar mass of dry air")
        R_atm = Variable("R_{atm}", 8.31447, "J/mol/K", "air specific heating value")
        TH = (g*M_atm/R_atm/L_atm).value

        constraints = [
            #h <= 20000*units.m,  # Model valid to top of troposphere
            T_sl >= T_atm + L_atm*h,     # Temp decreases w/ altitude
            # http://en.wikipedia.org/wiki/Density_of_air#Altitude
            rho <= p_sl*T_atm**(TH-1)*M_atm/R_atm/(T_sl**TH)]
        Model.__init__(self, None, constraints, **kwargs)

## Station Keeping Requirements

The minimum altitude needs to be 15,000 ft in order to reach the station keeping requirement of a 100 km diameter footprint.  In order to clear air traffic the minimum altitude needs to be at least 70,000 ft.  Graph below shows the wind distributions required to station keep at both 15,000 ft and 70,000 ft. 


In [ ]:
class StationKeeping(Model):
    def __init__(self, **kwargs):
        
        h_min = Variable('h_{min}', 15000, 'ft', 'minimum altitude')
        V_wind = Variable('V_{wind}', 10, 'm/s', 'wind speed')
        
        constraints = [h >= h_min,
                       V >= V_wind]
        Model.__init__(self, None, constraints, **kwargs)

## Overall sizing model

In [ ]:
class SolarHALE(Model):
    """High altitude long endurance solar UAV"""
    def __init__(self, **kwargs):
        """Setup method should return objective, list of constraints"""

        slf = SteadyLevelFlight()
        power = Power()
        weight = Weight()
        sk = StationKeeping()
        aero = Aero()
        atmosphere = Atmosphere()
        self.submodels = [slf, power, weight, sk, aero, atmosphere] 

        constraints = []
        lc = LinkedConstraintSet([self.submodels, constraints])

        objective = W

        Model.__init__(self, objective, lc, **kwargs)

if __name__ == "__main__":
    M = SolarHALE()
    sol = M.solve("mosek")

In [ ]:
M.solve("mosek")

In [ ]:
import numpy as np
M.substitutions.update({"h": ('sweep', np.linspace(15000, 50000, 4)),
                        "V_{wind}": ('sweep', np.linspace(10, 30, 5))})
sol = M.solve(solver="mosek", verbosity=0, skipsweepfailures=True)

In [ ]:
%matplotlib inline
from gpkit.interactive.plotting import contour_array
_ = contour_array(M, "V_{wind}", "h", ["MTOW", "b"])